6モデル（DT、RFC、LR、SVM、XGB、LGB）の結果を3モデル（RFC、XGB、LGB）で予測しなおし、その3モデルで出した結果をLGBモデルで予測し最終結果を出した。

In [52]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import sklearn.metrics
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier as DT #決定木　
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.linear_model import LogisticRegression #ロジスティック回帰
from sklearn import svm #サポートベクターマシン
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import robust_scale # 一次元配列はこれで直接。二次元配列はRobustScalerを使うことが多い
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

import scipy.stats

import seaborn as sns

import collections
import time

import lightgbm as lgb
import optuna

from xgboost import XGBClassifier

In [37]:
pd.set_option('display.max_rows', None)

データを変数に入れる

In [38]:
train = pd.read_table('train.tsv')
test = pd.read_csv('test.tsv',sep='\t')
sample = pd.read_csv('sample_submit.csv',header=None)

データを見る・データを整える

In [39]:
train = train.replace({"Y":{"<=50K" : 0, ">50K" : 1}})
train["Y"].value_counts()

0    12288
1     3992
Name: Y, dtype: int64

In [40]:
trainX = train.iloc[:,0:15]
y = train["Y"]
testX = test.copy()

In [41]:
# education = education-num より消去
train = train.drop(columns=["id","education"])
trainX = trainX.drop(columns=["id","education"])
testX = testX.drop(columns=["id","education"])

In [42]:
# native-countryについて、trainかtest片方にしかないものは？に変換してしまう。
indexNum = testX[~testX["native-country"].isin(trainX["native-country"].unique())].index
for i in indexNum:
    testX.iat[i,12] = "?"
    print(testX.iat[i,12])

?


In [43]:
# sexを数値化
trainX = trainX.replace({"sex":{"Female" : 0, "Male" : 1}})
testX = testX.replace({"sex":{"Female" : 0, "Male" : 1}})

In [44]:
pd.reset_option('display.max_rows')

In [45]:
#正規化（変な外れ値のなさそうな'age', 'education-num'）・robust（最大最小で2桁以上異なり外れ値もありそうな'fnlwgt'）
minmax_scaler = preprocessing.MinMaxScaler() # 二次元配列のもののみ（一次元配列：minmax_scale(対象となる配列))

trainX[['age', 'education-num']] = minmax_scaler.fit_transform(trainX[['age', 'education-num']]) 
testX[['age', 'education-num']] = minmax_scaler.fit_transform(testX[['age', 'education-num']]) 
trainX['fnlwgt'] = robust_scale(trainX['fnlwgt'])
testX['fnlwgt'] = robust_scale(testX['fnlwgt']) 

In [47]:
trainX = pd.get_dummies(trainX)
testX = pd.get_dummies(testX)

In [48]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(trainX, y, test_size=0.2, random_state = 100)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state = 100)
tests = [(X_test, y_test)]
print(trainX.shape, X_train.shape, X_valid.shape, X_test.shape)

(16280, 90) (10419, 90) (2605, 90) (3256, 90)


各モデルを作る

In [49]:
# model_DT
parameters = {"max_depth":list(range(5,14)), "min_samples_leaf":[46,47,48,49,50,51,52,53,54,55]}
clf_DT = DT()
model_DT = GridSearchCV(clf_DT, parameters, cv=5, scoring="roc_auc",n_jobs=-1,return_train_score=True)

model_DT.fit(X_train, y_train)
model_DT.best_params_

{'max_depth': 9, 'min_samples_leaf': 48}

In [50]:
# model_RFC
model_RFC = RandomForestClassifier(random_state=1234)
model_RFC.fit(X_train, y_train)

RandomForestClassifier(random_state=1234)

In [62]:
# model_LR

param_LR = {'C' : [0.01, 0.1, 1]}

clf_LR = LogisticRegression(random_state=1, solver='sag', max_iter = 10000, n_jobs=1)
model_LR = GridSearchCV(clf_LR, param_LR, cv=5)
model_LR.fit(X_train, y_train)

print("Best parameters : {}".format(model_LR.best_params_))
print("Best cross-validation score : {:.3f}".format(model_LR.best_score_))

Best parameters : {'C': 0.01}
Best cross-validation score : 0.794


In [17]:
# model_LGB

callbacks = []
callbacks.append(lgb.early_stopping(stopping_rounds=15))
callbacks.append(lgb.log_evaluation())

model_LGB = lgb.LGBMClassifier(boosting_type='goss', max_depth=6, random_state=0)
model_LGB.fit(X_train, y_train, eval_set=tests, callbacks=callbacks)

[1]	valid_0's binary_logloss: 0.511795
Training until validation scores don't improve for 15 rounds
[2]	valid_0's binary_logloss: 0.481821
[3]	valid_0's binary_logloss: 0.458024
[4]	valid_0's binary_logloss: 0.438747
[5]	valid_0's binary_logloss: 0.422328
[6]	valid_0's binary_logloss: 0.408078
[7]	valid_0's binary_logloss: 0.396088
[8]	valid_0's binary_logloss: 0.385551
[9]	valid_0's binary_logloss: 0.376185
[10]	valid_0's binary_logloss: 0.36848
[11]	valid_0's binary_logloss: 0.361077
[12]	valid_0's binary_logloss: 0.355859
[13]	valid_0's binary_logloss: 0.350556
[14]	valid_0's binary_logloss: 0.346252
[15]	valid_0's binary_logloss: 0.341767
[16]	valid_0's binary_logloss: 0.338374
[17]	valid_0's binary_logloss: 0.335306
[18]	valid_0's binary_logloss: 0.332723
[19]	valid_0's binary_logloss: 0.330275
[20]	valid_0's binary_logloss: 0.327873
[21]	valid_0's binary_logloss: 0.326079
[22]	valid_0's binary_logloss: 0.324424
[23]	valid_0's binary_logloss: 0.322592
[24]	valid_0's binary_logloss

LGBMClassifier(boosting_type='goss', max_depth=6, random_state=0)

In [18]:
# model_XGB
model_XGB = XGBClassifier(early_stopping_rounds=10)
model_XGB.fit(X_train, y_train, eval_set=tests, verbose=True)

[0]	validation_0-logloss:0.54449
[1]	validation_0-logloss:0.46501
[2]	validation_0-logloss:0.41468
[3]	validation_0-logloss:0.38220
[4]	validation_0-logloss:0.35960
[5]	validation_0-logloss:0.34401
[6]	validation_0-logloss:0.33277
[7]	validation_0-logloss:0.32501
[8]	validation_0-logloss:0.32006
[9]	validation_0-logloss:0.31710
[10]	validation_0-logloss:0.31420
[11]	validation_0-logloss:0.31108
[12]	validation_0-logloss:0.30944
[13]	validation_0-logloss:0.30702
[14]	validation_0-logloss:0.30683
[15]	validation_0-logloss:0.30477
[16]	validation_0-logloss:0.30358
[17]	validation_0-logloss:0.30217
[18]	validation_0-logloss:0.30206
[19]	validation_0-logloss:0.30053
[20]	validation_0-logloss:0.29875
[21]	validation_0-logloss:0.29906
[22]	validation_0-logloss:0.29860
[23]	validation_0-logloss:0.29864
[24]	validation_0-logloss:0.29833
[25]	validation_0-logloss:0.29830
[26]	validation_0-logloss:0.29837
[27]	validation_0-logloss:0.29730
[28]	validation_0-logloss:0.29691
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [19]:
# model_SVM
model_SVM = svm.SVC(gamma="scale")
model_SVM.fit(X_train, y_train)

SVC()

モデルごとの精度を見る

In [20]:
names = ["DT", "RFC", "LR", "LGB", "XGB", "SVM"]

classifiers = [
    model_DT,
    model_RFC,
    model_LR,
    model_LGB,
    model_XGB,
    model_SVM,
    ]

result = []

for name, clf in zip(names, classifiers):
    score1 = clf.score(X_train, y_train) # 正解率（train）の算出
    score2 = clf.score(X_test, y_test) # 正解率（test）の算出
    result.append([score1, score2]) # 結果の格納

In [21]:
df_result = pd.DataFrame(result, columns=['train', 'test'], index=names).sort_values('test', ascending=False)
df_result

,train,test
DT,0.916165,0.896870
XGB,0.896919,0.861794
LGB,0.887129,0.858108
RFC,1.000000,0.851658
SVM,0.801612,0.798833
LR,0.781745,0.771499


In [22]:
test_pred_DT = model_DT.predict(X_test)
test_pred_RFC = model_RFC.predict(X_test)
test_pred_LR = model_LR.predict(X_test)
test_pred_LGB = model_LGB.predict(X_test)
test_pred_XGB = model_XGB.predict(X_test)
test_pred_SVM = model_SVM.predict(X_test)

In [23]:
from sklearn.metrics import mean_squared_error
print ("モデル1の平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, test_pred_DT)))
print ("モデル1の平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, test_pred_RFC)))
print ("モデル1の平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, test_pred_LR)))
print ("モデル2の平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, test_pred_LGB)))
print ("モデル3の平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, test_pred_XGB)))
print ("モデル3の平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, test_pred_SVM)))

モデル1の平均2乗誤差: 0.1554
モデル1の平均2乗誤差: 0.1483
モデル1の平均2乗誤差: 0.2285
モデル2の平均2乗誤差: 0.1419
モデル3の平均2乗誤差: 0.1382
モデル3の平均2乗誤差: 0.2012


スタッキング（ブレンディング）

In [24]:
first_pred_DT = model_DT.predict(X_valid)
first_pred_RFC = model_RFC.predict(X_valid)
first_pred_LR = model_LR.predict(X_valid)
first_pred_LGB = model_LGB.predict(X_valid)
first_pred_XGB = model_XGB.predict(X_valid)
first_pred_SVM = model_SVM.predict(X_valid)

In [25]:
stack_pred = np.column_stack((first_pred_DT, first_pred_RFC, first_pred_LR, first_pred_LGB, first_pred_XGB, first_pred_SVM))
stack_pred

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [26]:
# 各モデルの検証データを積み重ねる
stack_test_pred = np.column_stack((test_pred_DT, test_pred_RFC, test_pred_LR, test_pred_LGB, test_pred_XGB, test_pred_SVM))

In [27]:
# メタモデルの学習 
meta_model1 = XGBClassifier(early_stopping_rounds=20)
meta_model2 = lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0)
meta_model3 = RandomForestClassifier(random_state=1234)
eval_set2 = [(stack_test_pred, y_test)]
meta_model1.fit(stack_pred, y_valid, eval_set = eval_set2, verbose=True)
meta_model2.fit(stack_pred, y_valid, eval_set = eval_set2, callbacks=callbacks)
meta_model3.fit(stack_pred, y_valid)

[0]	validation_0-logloss:0.54593
[1]	validation_0-logloss:0.46808
[2]	validation_0-logloss:0.42331
[3]	validation_0-logloss:0.39687
[4]	validation_0-logloss:0.38134
[5]	validation_0-logloss:0.37223
[6]	validation_0-logloss:0.36719
[7]	validation_0-logloss:0.36460
[8]	validation_0-logloss:0.36324
[9]	validation_0-logloss:0.36284
[10]	validation_0-logloss:0.36280
[11]	validation_0-logloss:0.36298
[12]	validation_0-logloss:0.36332
[13]	validation_0-logloss:0.36354
[14]	validation_0-logloss:0.36376
[15]	validation_0-logloss:0.36417
[16]	validation_0-logloss:0.36440
[17]	validation_0-logloss:0.36476
[18]	validation_0-logloss:0.36486
[19]	validation_0-logloss:0.36515
[20]	validation_0-logloss:0.36523
[21]	validation_0-logloss:0.36550
[22]	validation_0-logloss:0.36571
[23]	validation_0-logloss:0.36573
[24]	validation_0-logloss:0.36591
[25]	validation_0-logloss:0.36589
[26]	validation_0-logloss:0.36605
[27]	validation_0-logloss:0.36602
[28]	validation_0-logloss:0.36616
[29]	validation_0-loglos

RandomForestClassifier(random_state=1234)

In [28]:
meta1_first_pred = meta_model1.predict(stack_pred)
meta2_first_pred = meta_model2.predict(stack_pred)
meta3_first_pred = meta_model3.predict(stack_pred)
meta1_test_pred = meta_model1.predict(stack_test_pred)
meta2_test_pred = meta_model2.predict(stack_test_pred)
meta3_test_pred = meta_model3.predict(stack_test_pred)
print ("メタモデルの平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, meta1_test_pred)))
print ("メタモデルの平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, meta2_test_pred)))
print ("メタモデルの平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, meta3_test_pred)))

メタモデルの平均2乗誤差: 0.1382
メタモデルの平均2乗誤差: 0.1376
メタモデルの平均2乗誤差: 0.1382


In [29]:
stack_meta_pred = np.column_stack((meta1_first_pred, meta2_first_pred, meta3_first_pred))
stack_metatest_pred = np.column_stack((meta1_test_pred, meta2_test_pred, meta3_test_pred))

# 最終予測のメタモデルを作成
meta2_model = lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0)
eval_set3 = [(stack_metatest_pred, y_test)]
meta2_model.fit(stack_meta_pred, y_valid, eval_set = eval_set2, callbacks=callbacks)
meta_test_pred = meta2_model.predict(stack_metatest_pred)
print ("メタモデルの平均2乗誤差: {:.4f}".format(mean_squared_error(y_test, meta_test_pred)))

[1]	valid_0's binary_logloss: 0.518418
[2]	valid_0's binary_logloss: 0.494132
[3]	valid_0's binary_logloss: 0.475762
[4]	valid_0's binary_logloss: 0.4615
[5]	valid_0's binary_logloss: 0.450259
[6]	valid_0's binary_logloss: 0.441323
[7]	valid_0's binary_logloss: 0.434187
[8]	valid_0's binary_logloss: 0.42848
[9]	valid_0's binary_logloss: 0.423918
[10]	valid_0's binary_logloss: 0.420282
[11]	valid_0's binary_logloss: 0.417431
[12]	valid_0's binary_logloss: 0.41515
[13]	valid_0's binary_logloss: 0.413335
[14]	valid_0's binary_logloss: 0.411947
[15]	valid_0's binary_logloss: 0.410911
[16]	valid_0's binary_logloss: 0.410154
[17]	valid_0's binary_logloss: 0.409589
[18]	valid_0's binary_logloss: 0.409201
[19]	valid_0's binary_logloss: 0.40892
[20]	valid_0's binary_logloss: 0.408753
[21]	valid_0's binary_logloss: 0.408682
[22]	valid_0's binary_logloss: 0.40867
[23]	valid_0's binary_logloss: 0.408708
[24]	valid_0's binary_logloss: 0.408781
[25]	valid_0's binary_logloss: 0.408877
[26]	valid_0's 

In [30]:
production_pred_DT = model_DT.predict(testX)
production_pred_RFC = model_RFC.predict(testX)
production_pred_LR = model_LR.predict(testX)
production_pred_LGB = model_LGB.predict(testX)
production_pred_XGB = model_XGB.predict(testX)
production_pred_SVM = model_SVM.predict(testX)

In [31]:
stack_production_pred = np.column_stack((production_pred_DT, production_pred_RFC, production_pred_LR, production_pred_LGB, production_pred_XGB, production_pred_SVM))
pred1 = meta_model1.predict(stack_production_pred)
pred2 = meta_model2.predict(stack_production_pred)
pred3 = meta_model3.predict(stack_production_pred)

stack_metaproduction_pred = np.column_stack((pred1, pred2, pred3))
pred = meta2_model.predict(stack_metaproduction_pred)

In [32]:
pred_r = pd.DataFrame(data=pred, index=None, columns=None)
pred_r.value_counts()

0    13070
1     3211
dtype: int64

In [33]:
pred_r = pred_r.replace({0:{0 : "<=50K", 1 : ">50K"}})

In [34]:
sample[1] = pred_r

In [35]:
sample.to_csv("submit9_2.csv",index=None,header=None)